In [1]:
import numpy as np

In [2]:
delta_t = 0.05
t0 = 0.0
k0 = 0.1

def get_psi_dpsi_ddpsi(x,t):
    _psi = np.exp(1.0j * (k0 * x - 0.5*k0*k0*t))
    _dpsi = 1.0j * k0 * _psi
    _ddpsi = - k0 * k0 * _psi
    return (_psi, _dpsi, _ddpsi)

In [3]:
def prop_xx(xx_t, get_psi_dpsi_ddpsi, t, delta_t, log=False, g_zero_thres=1e-13, xx_diff_thres=1e-13):
    
    _t_next = t + delta_t
    
    _xx_t_k = xx_t

    while True:

        _psi, _dpsi, _ddpsi = get_psi_dpsi_ddpsi(_xx_t_k, _t_next)
        _g_k = xx_t - _xx_t_k + delta_t * (_dpsi / _psi).imag
        if log: print("xx_t_k: {:.16f} / g_k: {:.16f}".format(_xx_t_k, _g_k))

        if np.abs(_g_k) < g_zero_thres: break

        _psi_sq = (_psi.conj() * _psi).real
        _xx_t_k_next = _xx_t_k - ( (xx_t - _xx_t_k)*_psi_sq + delta_t * (_psi.conj()*_dpsi).imag ) / ( delta_t * (_psi.conj()*_ddpsi - _psi.conj() / _psi * (_dpsi*_dpsi)).imag - 1.0)
        assert np.abs(_xx_t_k_next - _xx_t_k) > xx_diff_thres
        _xx_t_k = _xx_t_k_next
    
    _xx_t_next = _xx_t_k
    
    return _xx_t_next

In [4]:
xx_t = 0.0
t = t0

for _t_idx in range(10):

    xx_t = prop_xx(xx_t, get_psi_dpsi_ddpsi, t, delta_t)
    t = t + delta_t
    
    print("t: {:.15f} / xx_t: {:.15f}".format(t,xx_t))

t: 0.050000000000000 / xx_t: 0.005000000000000
t: 0.100000000000000 / xx_t: 0.010000000000000
t: 0.150000000000000 / xx_t: 0.015000000000000
t: 0.200000000000000 / xx_t: 0.020000000000000
t: 0.250000000000000 / xx_t: 0.025000000000000
t: 0.300000000000000 / xx_t: 0.030000000000000
t: 0.350000000000000 / xx_t: 0.035000000000000
t: 0.400000000000000 / xx_t: 0.040000000000000
t: 0.450000000000000 / xx_t: 0.045000000000000
t: 0.500000000000000 / xx_t: 0.050000000000000
